In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler 


In [3]:
iris = datasets.load_iris() 
X,y = iris.data[50:,[1,2]] , iris.target[50:] 
le = LabelEncoder() 
y = le.fit_transform(y)

In [4]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.5,random_state=1,stratify=y)

In [5]:
#Train on 3 different classifiers 
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline 
import numpy as np

In [10]:
clf1 = LogisticRegression(penalty='l2',C=0.001,random_state=1)
clf2 = DecisionTreeClassifier(max_depth=1,criterion='entropy',random_state=0) 
clf3 = KNeighborsClassifier(n_neighbors=1,p=2,metric='minkowski')

In [22]:
pipe1 = Pipeline([('sc',StandardScaler()),('clf',clf1)])
pipe3 = Pipeline([('sc',StandardScaler()),('clf',clf3)]) 

clf_labels = ['Logistic Regression','Decision Tree','KNN'] 
print('10 fold cross validation :\n')
for clf , label in zip([pipe1,clf2, pipe3],clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print("ROC AUC : %0.2f (+/- %0.2f)[%s]" %(scores.mean(),scores.std(),label))

10 fold cross validation :

ROC AUC : 0.87 (+/- 0.17)[Logistic Regression]
ROC AUC : 0.89 (+/- 0.16)[Decision Tree]
ROC AUC : 0.88 (+/- 0.15)[KNN]


In [20]:
import sys
from MLAlgo.Ensemble import MajorityVoteClassifier 


In [21]:
mv_clf = MajorityVoteClassifier(classifiers=[pipe1,clf2,pipe3]) 
clf_labels += ['majority voting'] 
all_clf = [pipe1,clf2 , pipe3,mv_clf]
for clf , label in zip(all_clf,clf_labels):
    scores = cross_val_score(estimator=clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print("ROC AUC : %0.2f (+/- %0.2f)[%s]" %(scores.mean(),scores.std(),label))

ROC AUC : 0.87 (+/- 0.17)[Logistic Regression]
ROC AUC : 0.89 (+/- 0.16)[Decision Tree]
ROC AUC : 0.88 (+/- 0.15)[KNN]
ROC AUC : 0.94 (+/- 0.13)[majority voting]
